# Instalando dependencias

In [1]:
!nvidia-smi

Fri Jun 14 13:42:40 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 485, done.
remote: Counting objects: 100% (216/216), done.
remote: Compressing objects: 100% (125/125), done.
remote: Total 485 (delta 146), reused 124 (delta 91), pack-reused 269
Receiving objects: 100% (485/485), 134.51 KiB | 709.00 KiB/s, done.
Resolving deltas: 100% (248/248), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 715.7 kB/s eta 0:00:00
***********************************************************************
Woo! Your instance has a Tesla T4 GPU!
We will install the latest stable RAPIDS via pip 24.4.*!  Please stand by, should be quick...
***********************************************************************

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 GB 886.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 GB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.8/137.8 M

In [3]:
!pip install --upgrade tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 33.9 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      Successfully uninstalled tensorboard-2.15.2
  Attempting uninstall: keras
    Fo

In [4]:
import tensorflow as tf
tf.__version__

'2.16.1'

# Preprocesamiento

In [61]:
import pandas as pd
import os
import cudf
import gc
import cuml
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from sklearn.metrics.pairwise import cosine_similarity
#from cuml.recommendation import AlternatingLeastSquares as cuALS
#from cuml.preprocessing.model_selection import train_test_split

In [7]:
from google.colab import drive
drive.mount('/content/drive')

DIR = "/content/drive/MyDrive/MNA/BigData/Spotify_rec_sys/"
os.chdir(DIR)

Mounted at /content/drive


In [26]:
df = cudf.read_parquet("df.parquet")

df = df.dropna()

df = df.drop_duplicates()

df.columns = [col.replace('"', '').replace(' ','') for col in df.columns]


df.head()


,user_id,artistname,trackname,playlistname
0,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello,(The Angels Wanna Wear My) Red Shoes,HARD ROCK 2010
1,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello & The Attractions,"(What's So Funny 'Bout) Peace, Love And Unders...",HARD ROCK 2010
2,9cc0cfd4d7d7885102480dd99e7a90d6,Tiffany Page,7 Years Too Late,HARD ROCK 2010
3,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello & The Attractions,Accidents Will Happen,HARD ROCK 2010
4,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello,Alison,HARD ROCK 2010


In [29]:
# Crear índices únicos para cada columna
dims = {}

def df_dim(df, input_col):
    unique_vals = df[input_col].unique().reset_index(drop=True).reset_index()
    unique_vals.columns = [f"{input_col}_index", input_col]
    dims[input_col] = unique_vals

for col_name in df.columns:
    df_dim(df, col_name)

for dim in dims.values():
    print(dim.head(10))

for key, df_dim in dims.items():
    df_dim.to_parquet(f"{key}_dim.parquet")

   user_id_index                           user_id
0              0  9cc0cfd4d7d7885102480dd99e7a90d6
1              1  07f0fc3be95dcd878966b1f9572ff670
2              2  944c80d26922ae634d6ce445b1fdff7f
3              3  c5cdf06b5f1836006ef2a2fe4f5ff103
4              4  f3743cac98b7255c3c4a23be09dee7e6
5              5  c50566d83fba17b20697039d5824db78
6              6  83d7007a034208cdb13b7d3c29a02005
7              7  fa73fc4253c0a155bf953a9d43d2a46e
8              8  6d623100681e282b592c079594f03a89
9              9  650c4d63a819dbb77cc15a87f407039a
   artistname_index                        artistname
0                 0                    Elvis Costello
1                 1  Elvis Costello & The Attractions
2                 2                      Tiffany Page
3                 3                            Lissie
4                 4                    Paul McCartney
5                 5                          Joe Echo
6                 6                      The Breakers
7      

In [30]:

# Unir los índices al DataFrame original
newdf = df.copy()

for col_name in df.columns:
    newdf = newdf.merge(dims[col_name], on=col_name, how='left').drop(columns=[col_name])

newdf.columns = [f"{col_name}_index" for col_name in df.columns]

# Mostrar las primeras filas del nuevo DataFrame
print(newdf.head(10))

   user_id_index  artistname_index  trackname_index  playlistname_index
0              5              1216             2910                  38
1              5              1217             2911                  38
2              5              1218             2912                  38
3              5              1219             1941                  38
4              5              1220             2913                  38
5              5              1221             2914                  38
6              5               995             2524                  38
7              5               245             2915                  38
8              5              1222             2916                  38
9              5              1119             2917                  38


In [31]:
# Liberar memoria de los DataFrames dims en cuDF
dims.clear()
gc.collect()

382

In [32]:
print(len(newdf))
artist_counts = newdf.groupby('artistname_index').size().reset_index(name='reproductions')

# Filtrar los artistas con menos de 500 reproducciones
popular_artists = artist_counts[artist_counts['reproductions'] >= 500]['artistname_index']

# Filtrar el DataFrame original para mantener solo los artistas populares
filtered_df = newdf[newdf['artistname_index'].isin(popular_artists)]

# 2. Filtrar usuarios que no han escuchado al menos 10 artistas
user_artist_counts = filtered_df.groupby('user_id_index').size().reset_index(name='artist_count')
active_users = user_artist_counts[user_artist_counts['artist_count'] >= 10]['user_id_index']

filtered_df = filtered_df[filtered_df['user_id_index'].isin(active_users.to_pandas())]
print(len(filtered_df))

12868518
8472974


In [33]:
# Agrupar y contar reproducciones
counts_df = filtered_df.groupby(['user_id_index', 'artistname_index']).size().reset_index(name='reproductions')

# Mostrar las primeras filas del DataFrame agregado
print(counts_df.head(10))

   user_id_index  artistname_index  reproductions
0          14704             11087              6
1          15339              1333              1
2           2128             11727              2
3          13536              6203              2
4           9157               312              8
5           3061              3172              2
6          14005               217              1
7          12614              1172              1
8          12665              2304              3
9          13765               588              2


In [34]:
max_reproduction = counts_df['reproductions'].max()
min_reproduction = counts_df['reproductions'].min()

# Asegurarse de que max_reproduction y min_reproduction sean valores numéricos
max_reproduction = float(max_reproduction)
min_reproduction = float(min_reproduction)

# Crear una copia de counts_df para agregar la columna normalizada
normalized_pl_counts_df = counts_df.copy()

# Calcular y agregar la columna normalizada
normalized_pl_counts_df['normalized_reproduction'] = (normalized_pl_counts_df['reproductions'] - min_reproduction) / (max_reproduction - min_reproduction)

normalized_pl_counts_df['log_normalized_reproduction'] = np.log1p(normalized_pl_counts_df['normalized_reproduction'].to_pandas())

# Convertir de nuevo a cudf si es necesario
normalized_pl_counts_df = cudf.DataFrame.from_pandas(normalized_pl_counts_df)

# Mostrar las primeras filas del DataFrame resultante
print(normalized_pl_counts_df.head(10))

   user_id_index  artistname_index  reproductions  normalized_reproduction  \
0          14704             11087              6                 0.001495   
1          15339              1333              1                 0.000000   
2           2128             11727              2                 0.000299   
3          13536              6203              2                 0.000299   
4           9157               312              8                 0.002093   
5           3061              3172              2                 0.000299   
6          14005               217              1                 0.000000   
7          12614              1172              1                 0.000000   
8          12665              2304              3                 0.000598   
9          13765               588              2                 0.000299   

   log_normalized_reproduction  
0                     0.001494  
1                     0.000000  
2                     0.000299  
3        

In [35]:
#liberando memoria
del counts_df
newdf.to_parquet(f"clean_df.parquet")
del newdf
gc.collect()

199

In [36]:
unique_users = normalized_pl_counts_df['user_id_index'].unique()

# Dividir los usuarios en tres conjuntos: entrenamiento, validación y prueba
train_users, temp_users = train_test_split(unique_users, test_size=0.3, random_state=42)
val_users, test_users = train_test_split(temp_users, test_size=0.5, random_state=42)

# Filtrar los DataFrames originales en base a los conjuntos de usuarios
train_df = normalized_pl_counts_df[normalized_pl_counts_df['user_id_index'].isin(train_users)]
val_df = normalized_pl_counts_df[normalized_pl_counts_df['user_id_index'].isin(val_users)]
test_df = normalized_pl_counts_df[normalized_pl_counts_df['user_id_index'].isin(test_users)]

X_train_pd = train_df[['user_id_index', 'artistname_index']]
y_train_pd = train_df['log_normalized_reproduction']

X_test_pd = test_df[['user_id_index', 'artistname_index']]
y_test_pd = test_df['log_normalized_reproduction']

X_val_pd = val_df[['user_id_index', 'artistname_index']]
y_val_pd = val_df['log_normalized_reproduction']

# Convertir los DataFrames de train y test de nuevo a cudf
X_train = cudf.DataFrame.from_pandas(X_train_pd)
X_test = cudf.DataFrame.from_pandas(X_test_pd)
X_val = cudf.DataFrame.from_pandas(X_val_pd)

y_train = cudf.Series(y_train_pd)
y_test = cudf.Series(y_test_pd)
y_val = cudf.Series(y_val_pd)

# Liberar memoria
del X_train_pd, y_train_pd, X_test_pd, y_test_pd, X_val_pd, y_val_pd
gc.collect()


print(f"Train set size: {len(X_train)}")
print(f"Test set size: {len(X_val)}")
print(f"Val set size: {len(X_test)}")

Train set size: 1225176
Test set size: 278052
Val set size: 256978


#Entrenamiento de Red Neuronal Densa

In [40]:
num_users = X_train["user_id_index"].nunique()  # Count unique user IDs
num_items = X_train["artistname_index"].nunique()  # Count unique item IDs
embedding_dim = 100

In [41]:
print(num_users)
print(num_items)

10117
4039


In [42]:


# Entradas
user_input = layers.Input(shape=(1,))  # ID de usuario como un único entero
item_input = layers.Input(shape=(1,))  # ID de artista como un único entero

# Capas de embeddings
user_embedding = layers.Embedding(num_users, embedding_dim)(user_input)
item_embedding = layers.Embedding(num_items, embedding_dim)(item_input)

# Aplanar los embeddings (los embeddings de entrada tienen forma (batch_size, 1, embedding_dim))
user_embedding = layers.Flatten()(user_embedding)
item_embedding = layers.Flatten()(item_embedding)

# Concatenar los embeddings aplanados
merged = layers.concatenate([user_embedding, item_embedding])

# Capas ocultas y capa de salida
x = layers.Dense(128, activation="relu")(merged)
output = layers.Dense(1)(x)  # Capa de salida para predicciones

# Definir el modelo
model = Model(inputs=[user_input, item_input], outputs=output)
model.compile(loss="mse", optimizer="adam")

# Mostrar el resumen del modelo
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 1)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_3             │ (None, 1)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_2 (Embedding)   │ (None, 1, 100)         │      1,011,700 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_3 (Embedding)   │ (None, 1, 100)         │        403,900 │ input_layer_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_2 (Flatten)       │ (None, 100)            │              0 │ embedding_2[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_3 (Flatten)       │ (None, 100)            │              0 │ embedding_3[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_1             │ (None, 200)            │              0 │ flatten_2[0][0],       │
│ (Concatenate)             │                        │                │ flatten_3[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 128)            │         25,728 │ concatenate_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 1)              │            129 │ dense_2[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 1,441,457 (5.50 MB)

 Trainable params: 1,441,457 (5.50 MB)

 Non-trainable params: 0 (0.00 B)

In [43]:
model.fit(x=[X_train.to_numpy()[:, 0], X_train.to_numpy()[:, 1]], y=y_train.to_numpy(), epochs=10, validation_data=([X_val.to_numpy()[:, 0], X_val.to_numpy()[:, 1]], y_val.to_numpy()))

Epoch 1/10
38287/38287 ━━━━━━━━━━━━━━━━━━━━ 92s 2ms/step - loss: 3.1052e-05 - val_loss: 1.4710e-05
Epoch 2/10
38287/38287 ━━━━━━━━━━━━━━━━━━━━ 97s 3ms/step - loss: 2.1676e-05 - val_loss: 1.4307e-05
Epoch 3/10
38287/38287 ━━━━━━━━━━━━━━━━━━━━ 85s 2ms/step - loss: 1.8695e-05 - val_loss: 1.4605e-05
Epoch 4/10
38287/38287 ━━━━━━━━━━━━━━━━━━━━ 94s 2ms/step - loss: 2.1182e-05 - val_loss: 1.4926e-05
Epoch 5/10
38287/38287 ━━━━━━━━━━━━━━━━━━━━ 92s 2ms/step - loss: 2.3242e-05 - val_loss: 1.4565e-05
Epoch 6/10
38287/38287 ━━━━━━━━━━━━━━━━━━━━ 137s 2ms/step - loss: 2.0549e-05 - val_loss: 1.4324e-05
Epoch 7/10
38287/38287 ━━━━━━━━━━━━━━━━━━━━ 150s 2ms/step - loss: 2.1104e-05 - val_loss: 1.4278e-05
Epoch 8/10
38287/38287 ━━━━━━━━━━━━━━━━━━━━ 142s 2ms/step - loss: 2.1109e-05 - val_loss: 1.4711e-05
Epoch 9/10
38287/38287 ━━━━━━━━━━━━━━━━━━━━ 132s 2ms/step - loss: 2.0868e-05 - val_loss: 1.4469e-05
Epoch 10/10
38287/38287 ━━━━━━━━━━━━━━━━━━━━ 90s 2ms/step - loss: 2.0287e-05 - val_loss: 1.5410e-05


#Evaluando el modelo

In [45]:
from sklearn.metrics import mean_squared_error
predicted_reproductions = model.predict([X_train['user_id_index'].to_numpy(), X_train['artistname_index'].to_numpy()])

# Calcular el error cuadrático medio (MSE) para todo el conjunto de prueba
mse = mean_squared_error(y_train.to_numpy(), predicted_reproductions)

print(f"Mean Squared Error (MSE) X_train: {mse}")

38287/38287 ━━━━━━━━━━━━━━━━━━━━ 68s 2ms/step
Mean Squared Error (MSE) X_train: 2.0108257168224288e-05


In [46]:
from sklearn.metrics import mean_squared_error
pred_X_test = model.predict([X_test['user_id_index'].to_numpy(), X_test['artistname_index'].to_numpy()])

# Calcular el error cuadrático medio (MSE) para todo el conjunto de prueba
mse = mean_squared_error(y_test.to_numpy(), pred_X_test)

print(f"Mean Squared Error (MSE) X_test: {mse}")

8031/8031 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step
Mean Squared Error (MSE) X_test: 1.917968769151336e-05


#Evaluando predicciones

Para evaluar los ususarios que no fueron vistos durante el entrenamiento se crea matriz de similitud entre ussuarios según los artistas que escuchan.

Para un nuevo usuario se busca un usuario similar conocido y se brindan las recomendaciones de este último

## Matriz de similitud

In [117]:
combined_df = pd.concat([train_df.to_pandas(), test_df.to_pandas()])

In [118]:
interaction_matrix = combined_df.pivot(index='user_id_index', columns='artistname_index', values='log_normalized_reproduction').fillna(0)


In [119]:
user_similarity = cosine_similarity(interaction_matrix)

user_similarity_df = pd.DataFrame(user_similarity, index=interaction_matrix.index, columns=interaction_matrix.index)

In [120]:
user_similarity_df = cudf.DataFrame.from_pandas(user_similarity_df)

In [121]:
del user_similarity
gc.collect()

136

Para evaluar este modelo se utillizaran 3 metricas:
- Presision@K
- Mean Average Precision (MAP)
- Normalized Discounted Cumulative Gain (NDCG)

##Precision@K

In [163]:
import numpy as np
import cudf


def get_recommendations_for_new_user(user_id, user_similarity_df, model, num_items, top_k=10):

    similar_users = user_similarity_df[user_id].sort_values(ascending=False).head(top_k + 1).index.to_arrow().to_pylist()
    similar_users.remove(user_id)


    similar_user_predictions = np.zeros(num_items)
    for similar_user in similar_users:
        predictions = model.predict([np.array([similar_user] * num_items), np.arange(num_items)], verbose=0)
        similar_user_predictions += predictions.flatten()


    recommendations = np.argsort(similar_user_predictions)[::-1][:top_k]

    return recommendations


def precision_at_k(recommendations, actual_artists, k=10):
    num_correct = len(set(recommendations).intersection(actual_artists))
    precision = num_correct / k
    return precision


random_users_to_evaluate = np.random.choice(test_df['user_id_index'].unique().to_arrow().to_pylist(), size=100, replace=False)


k=10
precisions = []
for user_id in random_users_to_evaluate:
    actual_artists = test_df[test_df['user_id_index'] == user_id]['artistname_index'].to_arrow().to_pylist()
    recommendations = get_recommendations_for_new_user(user_id, user_similarity_df, model, num_items, top_k=k)
    precisions.append(precision_at_k(recommendations, actual_artists, k))

mean_precision_at_k = np.mean(precisions)
print(f"Mean Precision@{k}: {mean_precision_at_k}")


Mean Precision@10: 0.4331


##MAP@K (Mean Average Precision)

In [164]:
def average_precision_at_k(recommendations, actual_artists, k=10):
    num_correct = 0
    precision_sum = 0.0
    for i in range(k):
        artist_id = recommendations[i] if i < len(recommendations) else None
        if artist_id in actual_artists:
            num_correct += 1
            precision_sum += num_correct / (i + 1)  # Precision at i+1
    if not actual_artists:
        return 0.0
    return precision_sum / min(len(actual_artists), k)

def mean_average_precision_at_k(recommendations_list, actual_artists_list, k=10):
    total_map = 0.0
    for recommendations, actual_artists in zip(recommendations_list, actual_artists_list):
        ap = average_precision_at_k(recommendations, actual_artists, k)
        total_map += ap

    mean_map = total_map / len(recommendations_list)
    return mean_map

# Uso de la función
precisions = []
for user_id in random_users_to_evaluate:
    actual_artists = test_df[test_df['user_id_index'] == user_id]['artistname_index'].to_arrow().to_pylist()
    recommendations = get_recommendations_for_new_user(user_id, user_similarity_df, model, num_items, top_k=k).tolist()
    precisions.append((recommendations, actual_artists))

map_at_k = mean_average_precision_at_k([recs for recs, _ in precisions], [arts for _, arts in precisions], k)
print(f"Mean Average Precision@{k}: {map_at_k}")


Mean Average Precision@10: 0.1004


##NDGC@K (Normalized Discounted Cumulative Gain)

In [166]:
import math
def dcg_at_k(predichos, k):
    if not isinstance(predichos, list) or len(predichos) == 0:
        return 0.0

    dcg = predichos[0] if isinstance(predichos[0], (int, float)) else 0.0

    for i in range(1, min(k, len(predichos))):
        if isinstance(predichos[i], (int, float)):
            dcg += predichos[i] / math.log2(i + 1)

    return dcg

def ndcg_at_k(predichos, reales, k):
    dcg_predichos = dcg_at_k(predichos, k)
    dcg_reales = dcg_at_k(reales, k)
    if dcg_reales == 0:
        return 0.0
    else:
        ndcg = dcg_predichos / dcg_reales
        return ndcg

def ndcg_at_k_promedio_acumulado(predichos, reales, k, total_usuarios):

    global_sum = 0.0
    ndcg = ndcg_at_k(predichos, reales, k)
    global_sum += ndcg

    if total_usuarios == 0:
        return 0.0
    else:
        ndcg_promedio = global_sum / total_usuarios
        return ndcg_promedio


total_usuarios = 0
actual_artists_l = []
recommendations_l = []


for user_id in random_users_to_evaluate:
    actual_artists = test_df[test_df['user_id_index'] == user_id]['artistname_index'].to_arrow().to_pylist()
    recommendations = get_recommendations_for_new_user(user_id, user_similarity_df, model, num_items, top_k=k).tolist()


    actual_artists_l.append(actual_artists)
    recommendations_l.append(recommendations)
    total_usuarios += 1


ndcg_promedio_acumulado = ndcg_at_k_promedio_acumulado(actual_artists_l, recommendations_l, 10, total_usuarios)
print(f"NDCG@10 promedio acumulado = {ndcg_promedio_acumulado:.4f}")

NDCG@10 promedio acumulado = 0.0588


#Implementación

A continuacion se muestra cómo el modelo genera recomendaciones para un nuevo usuario:

In [176]:
#cargando el catalogo de artistas:
artist_dim = cudf.read_parquet("artistname_dim.parquet")
artist_dim.head()

,artistname_index,artistname
0,0,Elvis Costello
1,1,Elvis Costello & The Attractions
2,2,Tiffany Page
3,3,Lissie
4,4,Paul McCartney


In [207]:
test_df.head()

,user_id_index,artistname_index,reproductions,normalized_reproduction,log_normalized_reproduction
8,12665,2304,3,0.000598,0.000598
27,14875,1021,2,0.000299,0.000299
36,8271,763,18,0.005082,0.005069
37,7171,2569,7,0.001794,0.001792
39,1663,699,12,0.003288,0.003283


In [208]:
#Ususario de prueba
usr=14875
recs = get_recommendations_for_new_user(usr, user_similarity_df, model, num_items, top_k=10)

recs_df = cudf.DataFrame(recs,columns=['artistname_index'])

joined_data = cudf.merge(recs_df, artist_dim, on='artistname_index')

print(joined_data)

   artistname_index               artistname
0              3952                 Funky DL
1              3234           Rockabye Baby!
2                98  Wolfgang Amadeus Mozart
3              3268              Glenn Gould
4               706          Håkan Hellström
5               892          Lars Winnerbäck
6               210              Murray Gold
7               218   Vitamin String Quartet
8              2480                   Lucero
9               193                In Flames


In [209]:
hist=test_df[test_df['user_id_index']==usr]['artistname_index'].unique()

hist_df = cudf.DataFrame(hist,columns=['artistname_index'])

joined_data = cudf.merge(hist_df, artist_dim, on='artistname_index')

print(joined_data[:50])

    artistname_index               artistname
0               1791               Snoop Dogg
1                428             Frank Turner
2               1457        Frightened Rabbit
3                616        The Lonely Island
4                835            NEEDTOBREATHE
5                579            One Direction
6                583        Simon & Garfunkel
7               1736      Of Monsters and Men
8               4303              John Legend
9               2293             Dan Auerbach
10              1183                Parachute
11               189                    Gotye
12              2799                     Fun.
13              3138           Britney Spears
14              4771               Macklemore
15              2410       The Mountain Goats
16               636                 Ron Pope
17              1007                  Pitbull
18               773         The Chainsmokers
19               476  Macklemore & Ryan Lewis
20               477        A Grea